# Продвинутое машинное обучение: 
# Домашнее задание 3

Третье домашнее задание посвящено достаточно простой, но, надеюсь, интересной задаче, в которой потребуется творчески применить методы сэмплирования. Как и раньше, в качестве решения ожидается ссылка на jupyter-ноутбук на вашем github (или публичный, или с доступом для snikolenko); ссылку обязательно нужно прислать в виде сданного домашнего задания на портале Академии. Как всегда, любые комментарии, новые идеи и рассуждения на тему категорически приветствуются. 
В этом небольшом домашнем задании мы попробуем улучшить метод Шерлока Холмса.

Пользовался он для этого так называемым частотным методом: смотрел, какие буквы чаще встречаются в зашифрованных текстах, и пытался подставить буквы в соответствии с частотной таблицей: E — самая частая и так далее.
В этом задании мы будем разрабатывать более современный и продвинутый вариант такого частотного метода. В качестве корпусов текстов для подсчётов частот можете взять что угодно, но для удобства вот вам “Война и мир” по-русски и по-английски:
https://www.dropbox.com/s/k23enjvr3fb40o5/corpora.zip 

1. Реализуйте базовый частотный метод по Шерлоку Холмсу:
- подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
- возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
- расшифруйте их таким частотным методом.

In [1]:
import re
import numpy as np
import zipfile
import random
import collections
import warnings
warnings.filterwarnings('ignore')

from Levenshtein import ratio

In [2]:
# чтение zip архива
corpora_zip = zipfile.ZipFile('corpora.zip')

# загрузка файла AnnaKarenina
with corpora_zip.open('AnnaKarenina.txt', 'r') as readFile:
    karenina_text = readFile.read().decode('utf8')
# загрузка файла AnnaKarenina
with corpora_zip.open('WarAndPeace.txt', 'r') as readFile:
    war_peace_text = readFile.read().decode('utf8')
# загрузка файла AnnaKarenina
with corpora_zip.open('WarAndPeaceEng.txt', 'r') as readFile:
    war_peace_eng_text = readFile.read().decode('utf8')

In [3]:
def preprocessing(text, language):
    """Обработка текста."""
    
    text = text.lower()
    if language == 'russian':
        text = re.sub(r'[^а-яА-ЯыъьёЁ]', " ", text)
    else:
        text = re.sub(r'[^a-zA-Z]', " ", text)
    text = re.sub('\s+', ' ', text)
    text = text.strip()
    
    return text

In [4]:
# обрабатываем все файлы
karenina_text = preprocessing(karenina_text, language='russian')
war_peace_text = preprocessing(war_peace_text, language='russian')
war_peace_eng_text = preprocessing(war_peace_eng_text, language='english')

In [5]:
# подсчитаем частоты букв по корпусам
val_freq = collections.Counter(karenina_text)
karenina_freq = {key: val for key, val in val_freq.items() if key in set(karenina_text)}

In [6]:
# берем рандомный текст и делаем перестановку символов
random_text_1 = """Левин думал о евангельском изречении не потому, чтоб он считал себя премудрым. Он не считал себя премудрым, но не мог не знать, что он был умнее жены и Агафьи Михайловны, и не мог не знать того, что, когда он думал о смерти, он думал всеми силами души. Он знал тоже, что многие мужские большие умы, мысли которых об этом он читал, думали об этом и не знали одной сотой того, что знала об этом его жена и Агафья Михайловна. Как ни различны были эти две женщины, Агафья Михайловна и Катя, как ее называл брат Николай и как теперь Левину было особенно приятно называть ее, они в этом были совершенно похожи. Обе несомненно знали, что такое была жизнь и что такое была смерть, и хотя никак не могли ответить и не поняли бы даже тех вопросов, которые представлялись Левину, обе не сомневались в значении этого явления и совершенно одинаково, не только между собой, но разделяя этот взгляд с миллионами людей, смотрели на это. Доказательство того, что они знали твердо, что такое была смерть, состояло в том, что они, ни секунды не сомневаясь, знали, как надо действовать с умирающими, и не боялись их. Левин же и другие, хотя и многое могли сказать о смерти, очевидно не знали, потому что боялись смерти и решительно не знали, что надо делать, когда люди умирают. Если бы Левин был теперь один с братом Николаем, он бы с ужасом смотрел на него и еще с бóльшим ужасом ждал, и больше ничего бы не умел сделать.
Мало того, он не знал, что говорить, как смотреть, как ходить. Говорить о постороннем ему казалось оскорбительным, нельзя; говорить о смерти, о мрачном – тоже нельзя. Молчать – тоже нельзя. «Смотреть – он подумает, что я изучаю его, боюсь; не смотреть – он подумает, что я о другом думаю. Ходить на цыпочках – он будет недоволен; на всю ногу – совестно». Кити же, очевидно, не думала и не имела времени думать о себе; она думала о нем, потому что знала что-то, и все выходило хорошо. Она и про себя рассказывала и про свою свадьбу, и улыбалась, и жалела, и ласкала его, и говорила о случаях выздоровления, и все выходило хорошо; стало быть, она знала. Доказательством того, что деятельность ее и Агафьи Михайловны была не инстинктивная, животная, неразумная, было то, что, кроме физического ухода, облегчения страданий, и Агафья Михайловна и Кити требовали для умирающего еще чего-то такого, более важного, чем физический уход, и чего-то такого, что не имело ничего общего с условиями физическими. Агафья Михайловна, говоря об умершем старике, сказала: «Что ж, слава Богу, причастили, соборовали, дай Бог каждому так умереть». Катя точно так же, кроме всех забот о белье, пролежнях, питье, в первый же день успела уговорить больного в необходимости причаститься и собороваться."""
random_text_2 = """Вернувшись от больного на ночь в свои два нумера, Левин сидел, опустив голову, не зная, что делать. Не говоря уже о том, чтоб ужинать, устраиваться на ночлег, обдумывать, что они будут делать, он даже и говорить с женою не мог: ему совестно было. Кити же, напротив, была деятельнее обыкновенного. Она даже была оживленнее обыкновенного. Она велела принести ужинать, сама разобрала вещи, сама помогла стлать постели и не забыла обсыпать их персидским порошком. В ней было возбуждение и быстрота соображения, которые появляются у мужчин пред сражением, борьбой, в опасные и решительные минуты жизни, те минуты, когда раз навсегда мужчина показывает свою цену и то, что все прошедшее его было не даром, а приготовлением к этим минутам.
Все дело спорилось у нее, и еще не было двенадцати, как все вещи были разобраны чисто, аккуратно, как-то так особенно, что нумер стал похож на дом, на ее комнаты: постели постланы, щетки, гребни, зеркальца выложены, салфеточки постланы.
Левин находил, что непростительно есть, спать, говорить даже теперь, и чувствовал, что каждое движение его было неприлично. Она же разбирала щеточки, но делала все это так, что ничего в этом оскорбительного не было.
Есть, однако, они ничего не могли, и долго не могли заснуть, и даже долго не ложились спать."""

# предобработаем их
random_text_1 = preprocessing(random_text_1, language='russian')
random_text_2 = preprocessing(random_text_2, language='russian')

# перемешиваем и получаем рандомные значений
shuffle_text = sorted(list(set(karenina_text)))
# перемешиваем
random.shuffle(shuffle_text)
encoding_text = {key: value for key, value in zip(sorted(list(set(karenina_text))), shuffle_text)}

# кодируем текст
encoding_text_1 = ''.join(encoding_text[txt] for txt in random_text_1)
encoding_text_2 = ''.join(encoding_text[txt] for txt in random_text_2)
print(f"Оригинальный текст: {random_text_1[:50]}\nЗакодированный текст: {encoding_text_1[:50]}")

Оригинальный текст: левин думал о евангельском изречении не потому что
Закодированный текст: фрлпнгтекжфгэгрлжньрфчвхэкгпюмрырнппгнргдэзэкегызэ


In [7]:
# декодируем последовательность
def decode_text(text, dict_text):
    """Декодирование последовательности."""
    
    val_freq = collections.Counter(text)
    text_freq = {key: val for key, val in val_freq.items() if key in set(karenina_text)}
    sort_text_freq = dict(val_freq.most_common())
    letter_pairs = {k: v for k, v in zip(sort_text_freq, dict_text)}
    decoded_text = ''.join(letter_pairs[let] for let in text)
    
    return decoded_text

In [8]:
# формируем словарь 
dict_text = dict(val_freq.most_common())
decoding_text_1 = decode_text(encoding_text_1, dict_text)
decoding_text_2 = decode_text(encoding_text_2, dict_text)
print(f"Пример 1:\nОригинальный текст:{random_text_1[:50]}\nДекодированный текст:{decoding_text_1[:50]}")
print(f"Сходство текста по метрике Левинштейна для 1 примера: {ratio(random_text_1, decoding_text_1)}")
print(f"\nПример 2:\nОригинальный текст:{random_text_2[:50]}\nДекодированный текст:{decoding_text_2[:50]}")
print(f"Сходство текста по метрике Левинштейна для 2 примера: {ratio(random_text_2, decoding_text_2)}")

Пример 1:
Оригинальный текст:левин думал о евангельском изречении не потому что
Декодированный текст:серни мьлас о ераияесдвуол нбкеыеинн ие жотоль ыто
Сходство текста по метрике Левинштейна для 1 примера: 0.5097503900156006

Пример 2:
Оригинальный текст:вернувшись от больного на ночь в свои два нумера л
Декодированный текст:вераявйилм от досмаого ан аочм в лвои квн аяперн с
Сходство текста по метрике Левинштейна для 2 примера: 0.5967346938775511


### Вывод: фразы распознаются, но довольно трудно понять смысл.

2. Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
- подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
- проведите тестирование аналогично п.1, но при помощи биграмм.

In [9]:
# формируем словарь для биграмм
def to_bigramm(text):
    """Формирование биграмм."""
    
    bigram_freq = {}
    len_text = len(text)
    for iterr in range(len_text - 1):
        # формируем биграмы по 2 символам
        bigram = text[iterr:iterr + 2]
        # смотрим их наличие в словаре
        if bigram_freq.get(bigram, None):
            bigram_freq[bigram] += 1
        else:
            bigram_freq[bigram] = 1
    
    # формируем словарь с биграмми в виде ключа и значениями
    bigram_freq = dict(sorted(bigram_freq.items(), key=lambda val: val[1], reverse=True))
    
    return bigram_freq

# биграмы для всего текста
bigram_freq = to_bigramm(karenina_text)

In [10]:
def decode_bigram(text, dict_text):
    """Функция для декодирования тестовых текстов."""

    # биграмы для входного текста
    text_freq = to_bigramm(text)
    decoder_text = {key: value for key, value in zip(text_freq, dict_text)}
    len_text = len(text)
    text_new = []
    for iterr in range(0, len_text - 1, 2):
        text_new.append(text[iterr:iterr + 2])
    # формируем декодированный текст
    text = ''.join(decoder_text[bigram] for bigram in text_new)

    return text

In [11]:
# декодируем текст
decoding_bigram_1 = decode_bigram(encoding_text_1, bigram_freq)
decoding_bigram_2 = decode_bigram(encoding_text_2, bigram_freq)
print(f"Пример 1:\nОригинальный текст:{random_text_1[:50]}\nДекодированный текст по биграммам:{decoding_bigram_1[:50]}")
print(f"Сходство текста по метрике Левинштейна для 1 примера: {ratio(random_text_1, decoding_bigram_1)}")
print(f"\nПример 2:\nОригинальный текст:{random_text_2[:50]}\nДекодированный текст по биграммам:{decoding_bigram_2[:50]}")
print(f"Сходство текста по метрике Левинштейна для 2 примера: {ratio(random_text_2, decoding_bigram_2)}")

Пример 1:
Оригинальный текст:левин думал о евангельском изречении не потому что
Декодированный текст по биграммам:деасраетчтеро быопач мженитоехбин ув на ити  г ки 
Сходство текста по метрике Левинштейна для 1 примера: 0.4079563182527301

Пример 2:
Оригинальный текст:вернувшись от больного на ночь в свои два нумера л
Декодированный текст по биграммам:у внруше э вс нытенаста  ннаош ть  ми нн н ч яниэт
Сходство текста по метрике Левинштейна для 2 примера: 0.4115965700285831


### Вывод: результаты стали хуже, кодировалось по одному методу, а декодируется по другому

3. Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
- предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
- реализуйте и протестируйте его, убедитесь, что результаты улучшились.

In [12]:
# MCMC-сэмплирование
# все символы текста
alphabet = ''.join(sorted(list(set(karenina_text))))
val_alphabet = len(alphabet)
# преобразуем символы в индекс
char_to_index = {val: key for key, val in enumerate(alphabet)}

In [13]:
def train_mcc(text, char_to_index, alphabet, val_alphabet):
    """Обучение mcmc."""
    
    # формируем матрицу
    matrix_val = np.zeros((val_alphabet, val_alphabet))
    len_text = len(text)
    for iterr in range(len_text - 1):
        matrix_val[char_to_index[text[iterr]], char_to_index[text[iterr + 1]]] += 1
    matrix_val = np.clip(matrix_val, 1, None)
    matrix_val = (np.log(matrix_val).T - np.log(matrix_val.sum(axis=1))).T
    
    return matrix_val

# определяем матрицу признаков
matrix_val = train_mcc(karenina_text, char_to_index, alphabet, val_alphabet)

In [14]:
def log_likelihood(text, alphabet, matrix_val, char_to_index, transpose):
    """Определяем функцию правдоподобия."""

    # определяем текст
    translation = str.maketrans(alphabet, ''.join(transpose))
    text = text.translate(translation)

    # считаем логарифм правдоподобия
    likelihood = 0
    for iterr in range(len(text) - 1):
        likelihood += matrix_val[char_to_index[text[iterr]], char_to_index[text[iterr + 1]]]

    return likelihood
    
def decode_mcmc(text, alphabet, matrix_val, char_to_index, iteration):
    """Функция декодирования по MCMC."""
    
    transpose = np.array(list(alphabet))
    random.shuffle(transpose)
    value_likelihood = log_likelihood(text, alphabet, matrix_val, char_to_index, transpose)
    best_value = value_likelihood
    best_transpose = transpose.copy()
    for i in range(iteration):
        pass_index = random.sample(range(len(alphabet)), 2)
        transpose[pass_index[0]], transpose[pass_index[1]] = transpose[pass_index[1]], transpose[pass_index[0]]
        new_likelihood = log_likelihood(text, alphabet, matrix_val, char_to_index, transpose)
        if new_likelihood >= value_likelihood:
            value_likelihood = new_likelihood
            if new_likelihood > best_value:
                best_value = new_likelihood
                best_transpose = transpose.copy()
        else:
            if random.random() < np.exp(new_likelihood - value_likelihood):
                value_likelihood = new_likelihood
            else:
                transpose[pass_index[0]], transpose[pass_index[1]] = transpose[pass_index[1]], transpose[pass_index[0]]

    # определяем текст
    translation_out = str.maketrans(alphabet, ''.join(best_transpose))
    answer = text.translate(translation_out)

    return answer

In [19]:
# декодируем последовательность
decoded_mcc_text_1 = decode_mcmc(encoding_text_1, alphabet, matrix_val, char_to_index, iteration=200000)
print(f"Пример 1:\nОригинальный текст:{random_text_1[:50]}\nДекодированный текст по mcmc:{decoded_mcc_text_1[:50]}")
print(f"Сходство текста по метрике Левинштейна для 1 примера: {ratio(random_text_1, decoded_mcc_text_1)}")

Пример 1:
Оригинальный текст:левин думал о евангельском изречении не потому что
Декодированный текст по mcmc:левин думал о евангельском изречении не потому что
Сходство текста по метрике Левинштейна для 1 примера: 1.0


In [20]:
# декодируем последовательность
decoded_mcc_text_2 = decode_mcmc(encoding_text_2, alphabet, matrix_val, char_to_index, iteration=200000)
print(f"Пример 2:\nОригинальный текст:{random_text_2[:50]}\nДекодированный текст по mcmc:{decoded_mcc_text_2[:50]}")
print(f"Сходство текста по метрике Левинштейна для 1 примера: {ratio(random_text_2, decoded_mcc_text_2)}")

Пример 2:
Оригинальный текст:вернувшись от больного на ночь в свои два нумера л
Декодированный текст по mcmc:вернувшись от больного на ночь в свои два нумера л
Сходство текста по метрике Левинштейна для 1 примера: 1.0


4. Расшифруйте сообщение:
←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏
Или это (они одинаковые, второй вариант просто на случай проблем с юникодом):
დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ


In [21]:
# текстовое сообщение
text_message = "დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ"
# преобразуем последовательность
list_message_symbol = list(set(text_message))
decoder_alphabet = {key: value for key, value in zip(list_message_symbol, alphabet)}
encode_message = ''.join(decoder_alphabet[char] for char in text_message)
# декодируем сообщение
message_decode = decode_mcmc(encode_message, alphabet, matrix_val, char_to_index, iteration=200000)
print(f"Пример 2:\nОригинальный текст:{text_message}\nДекодированный текст по mcmc:{message_decode}")

Пример 2:
Оригинальный текст:დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ
Декодированный текст по mcmc:если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж


5. Бонус: а что если от биграмм перейти к триграммам (тройкам букв) или даже больше? Улучшатся ли результаты? Когда улучшатся, а когда нет? Чтобы ответить на этот вопрос эмпирически, уже может понадобиться погенерировать много тестовых перестановок и последить за метриками, глазами может быть и не видно.

In [22]:
# формируем словарь для биграмм
def n_gramm(text, n):
    """Формирование n - грамм."""
    
    ngram_freq = {}
    len_text = len(text)
    for iterr in range(len_text - 1):
        # формируем биграмы по n символам
        bigram = text[iterr:iterr + n]
        # смотрим их наличие в словаре
        if ngram_freq.get(bigram, None):
            ngram_freq[bigram] += 1
        else:
            ngram_freq[bigram] = 1
    
    # формируем словарь с биграмми в виде ключа и значениями
    ngram_freq = dict(sorted(ngram_freq.items(), key=lambda val: val[1], reverse=True))
    
    return ngram_freq

In [23]:
def decode_ngram(text, dict_text, n):
    """Функция для декодирования тестовых текстов."""

    # n-грамы для входного текста
    text_freq = n_gramm(text, n)
    decoder_text = {key: value for key, value in zip(text_freq, dict_text)}
    len_text = len(text)
    text_new = []
    for iterr in range(0, len_text - 1, n):
        text_new.append(text[iterr:iterr + n])
    # формируем декодированный текст
    text = ''.join(decoder_text[bigram] for bigram in text_new)

    return text

In [24]:
# проходим по различным n-граммам и выводим результаты метрик
for iterr in range(2, 6):
    # n-грамы для всего текста
    ngram_freq = n_gramm(karenina_text, iterr)
    decoding_ngram_1 = decode_ngram(encoding_text_1, ngram_freq, n=iterr)
    decoding_ngram_2 = decode_ngram(encoding_text_2, ngram_freq, n=iterr)
    print(f"Количество n-грамм: {iterr}")
    print(f"Пример 1:\nОригинальный текст:{random_text_1[:50]}\nДекодированный текст по биграммам:{decoding_ngram_1[:50]}")
    print(f"Сходство текста по метрике Левинштейна для 1 примера: {ratio(random_text_1, decoding_ngram_1)}")
    print(f"Пример 2:\nОригинальный текст:{random_text_2[:50]}\nДекодированный текст по биграммам:{decoding_ngram_2[:50]}")
    print(f"Сходство текста по метрике Левинштейна для 2 примера: {ratio(random_text_2, decoding_ngram_2)}\n")

Количество n-грамм: 2
Пример 1:
Оригинальный текст:левин думал о евангельском изречении не потому что
Декодированный текст по биграммам:деасраетчтеро быопач мженитоехбин ув на ити  г ки 
Сходство текста по метрике Левинштейна для 1 примера: 0.4079563182527301
Пример 2:
Оригинальный текст:вернувшись от больного на ночь в свои два нумера л
Декодированный текст по биграммам:у внруше э вс нытенаста  ннаош ть  ми нн н ч яниэт
Сходство текста по метрике Левинштейна для 2 примера: 0.4115965700285831

Количество n-грамм: 3
Пример 1:
Оригинальный текст:левин думал о евангельском изречении не потому что
Декодированный текст по биграммам:о та и этсь  дуложе уобе в сли нувое и алабылскибе
Сходство текста по метрике Левинштейна для 1 примера: 0.41567557028660557
Пример 2:
Оригинальный текст:вернувшись от больного на ночь в свои два нумера л
Декодированный текст по биграммам:ую ки а дее у нь обылна овоомуатьговствь итакожее 
Сходство текста по метрике Левинштейна для 2 примера: 0.4189465087790935

К

### Вывод:
Судя по результатам, лучше всего подходят n-граммы, которые равны 3 и 4, оценка проводилась по метрике Левинштейна.

6. Бонус: какие вы можете придумать применения для этой модели? Пляшущие человечки ведь не так часто встречаются в жизни (хотя встречаются! и это самое потрясающее во всей этой истории, но об этом я расскажу потом).

### Ответ:

Как мне кажется, главное применение может быть для людей, которые только учатся говорить после болезни или маленьких детей, которым сложно говорить, Можно преобразовывать их слова и фразы в текст и дальше применять данный подход. Либо в ситуации, когда сбивается кодировка для восстановления исходного текста.